In [1]:
import pyomo.environ as pe  # Pyomo environment


In [2]:

from idaes.core import FlowsheetBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_lmtd_callback
from idaes.models.properties import iapws95
import numpy as np


In [3]:
# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.params = iapws95.Iapws95ParameterBlock()

temperatures taken from Cinti
,2020

In [4]:
# Provided data in Celsius, convert to Kelvin
T_hot_inlet_C = 662.30
T_hot_outlet_C = 192.60
T_cold_inlet_C = 20.00
T_cold_outlet_C = 496.94


T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin


Check these!

In [5]:

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.039  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air at constant pressure in kJ/kg·K


In [6]:

# Reference temperature (for simplicity, assume 0°C = 273.15 K)
T_ref = 273.15

Look into molar flow rates

In [7]:
# Flow rate (Assuming constant flow rate of 10 mol/s)
flow_mol_hot = 10  # mol/s
flow_mol_cold = 10  # mol/s

In [8]:
# Convert molar flow to mass flow (assuming ideal gas behavior)
M_hot = 28.0134  # Molar mass of N2 (kg/kmol)
M_cold = 28.97  # Molar mass of air (kg/kmol)

In [9]:
mass_flow_hot = flow_mol_hot * M_hot / 1000  # Convert to kg/s
mass_flow_cold = flow_mol_cold * M_cold / 1000  # Convert to kg/s

In [10]:
# Print the mass flow rates for hot and cold streams
print(f"Mass flow rate of hot stream: {mass_flow_hot:.4f} kg/s")
print(f"Mass flow rate of cold stream: {mass_flow_cold:.4f} kg/s")

Mass flow rate of hot stream: 0.2801 kg/s
Mass flow rate of cold stream: 0.2897 kg/s


air flow rate in line with Cinti 2020 = 228.28 Nl/s ca. 295.2 g/s

In [12]:
# Compute heat duty (Q)
Q_hot = mass_flow_hot * Cp_hot * (T_hot_inlet - T_hot_outlet)  # kJ/s = kW
Q_cold = mass_flow_cold * Cp_cold * (T_cold_outlet - T_cold_inlet)  # kJ/s = kW

In [13]:
# Print Q_hot and Q_cold values
print(f"Q_hot: {Q_hot:.2f} kW")
print(f"Q_cold: {Q_cold:.2f} kW")

Q_hot: 136.71 kW
Q_cold: 138.86 kW


In [14]:
# Ensure heat balance is satisfied
Q = min(Q_hot, Q_cold) * 1000  # Convert kW to W

In [15]:
# Print the computed Q value
print(f"Computed Q: {Q/1000:.2f} kW")

Computed Q: 136.71 kW


In [16]:
# Compute LMTD
delta_T1 = T_hot_inlet - T_cold_outlet
delta_T2 = T_hot_outlet - T_cold_inlet
LMTD = (delta_T1 - delta_T2) / np.log(delta_T1 / delta_T2)
# Print the computed LMTD
print(f"Computed LMTD: {LMTD:.2f} K")

Computed LMTD: 168.95 K


In [17]:
# Fixed overall heat transfer coefficient from Cinti et al 2020
U = 30  # W/m²·K

In [18]:

# Compute heat exchanger area
A = Q / (U * LMTD)
print(f"Computed Heat Exchanger Area: {A:.2f} m²")

Computed Heat Exchanger Area: 26.97 m²


From here down cost calculation

Using cost function from Denver Chaddie 2011

In [19]:
# Assuming A is already defined as the heat exchanger area in m²
cost_2011 = 130 * (A / 0.093) ** 0.78
print(f"Computed Cost using 2011 cost function: EUR {cost_2011:.2f}")


Computed Cost using 2011 cost function: EUR 10830.23


We are doing this to follow the 5 years rule!!!!!

Price 2020 from market value alfa laval AlfaNova 76-100H

In [20]:
price_2020=9608 # EUR

In [21]:
price_ratio = price_2020/cost_2011

In [22]:
cost_2020=(130*price_ratio) * (A / 0.093) ** 0.78
print(f"Updated parameter {130*price_ratio:.2f}")

Updated parameter 115.33


In [23]:
# CEPCI values
CEPCI_base = 596.2  #2020  
CEPCI_current = 790.7 #2024

# Updated cost with CEPCI scaling
updated_cost = cost_2020 * (CEPCI_current / CEPCI_base)
print(f"Updated Cost to 2024: EUR {updated_cost:.2f}")


Updated Cost to 2024: EUR 12742.44
